In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(5,5,3))
x = Conv2D(2, 3)(inputs)
model = Model(inputs, x)

In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 5, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 3, 3, 2)        │            56 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56 (224.00 B)

 Trainable params: 56 (224.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.weights

[<KerasVariable shape=(3, 3, 3, 2), dtype=float32, path=conv2d/kernel>,
 <KerasVariable shape=(2,), dtype=float32, path=conv2d/bias>]

In [6]:
X = np.random.rand(1,5,5,3)
X

array([[[[0.2370103 , 0.89009444, 0.29281722],
         [0.88789716, 0.98268883, 0.56734385],
         [0.25664565, 0.36709193, 0.99686411],
         [0.92953875, 0.37995756, 0.36208754],
         [0.90235967, 0.85445041, 0.21331367]],

        [[0.81557373, 0.96464312, 0.31798285],
         [0.87162898, 0.55535123, 0.61857349],
         [0.20663315, 0.93180219, 0.27442014],
         [0.25269897, 0.17672735, 0.57902121],
         [0.41428354, 0.77645207, 0.48772502]],

        [[0.18615967, 0.07785182, 0.43524021],
         [0.040587  , 0.77615149, 0.13599362],
         [0.27840056, 0.61817123, 0.49579618],
         [0.16599803, 0.74690162, 0.72845412],
         [0.18505791, 0.92419726, 0.06217002]],

        [[0.99612826, 0.754579  , 0.83441526],
         [0.49431173, 0.2887831 , 0.30215832],
         [0.28381229, 0.71243764, 0.70628207],
         [0.45661526, 0.93958671, 0.45489822],
         [0.70917471, 0.01951241, 0.06535602]],

        [[0.61630486, 0.4752299 , 0.37425495],
     

In [7]:
y = model.predict(X)
y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[[[ 0.90196264,  0.01918238],
         [ 0.5806077 ,  0.54100686],
         [-0.05500734,  0.44589862]],

        [[ 0.34604475,  0.27477172],
         [ 0.6216948 ,  0.23142728],
         [ 0.69993114,  0.20226912]],

        [[ 0.7192656 ,  0.8509364 ],
         [ 0.8902939 , -0.12750588],
         [ 0.5513488 ,  0.7790845 ]]]], dtype=float32)

In [8]:
X_in = [[[int(X[0][i][j][k]*2**10) for k in range(3)] for j in range(5)] for i in range(5)]
X_in_unchanged = [[[str(X[0][i][j][k]) for k in range(3)] for j in range(5)] for i in range(5)]
weights = [[[[int(model.weights[0].numpy()[i][j][k][l]*2**10) for l in range(2)] for k in range(3)] for j in range(3)] for i in range(3)]
weights_unchanged = [[[[str(model.weights[0].numpy()[i][j][k][l]) for l in range(2)] for k in range(3)] for j in range(3)] for i in range(3)]
bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]
bias_unchanged = [str(model.weights[1].numpy()[i]) for i in range(2)]

In [9]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k]) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l]) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i]) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                out[i][j][m] = str(out[i][j][m] // n)
    return Input, Weights, Bias, out

In [10]:
X_in, weights, bias, out = Conv2DInt(5, 5, 3, 2, 3, 1, 2**10, X_in, weights, bias)
out

[[['922', '17'], ['592', '552'], ['-57', '453']],
 [['352', '280'], ['635', '235'], ['714', '207']],
 [['734', '868'], ['909', '-132'], ['563', '795']]]

In [11]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "in_unchanged": X_in_unchanged,
    "weights_unchanged": weights_unchanged,
    "bias_unchanged": bias_unchanged
}

In [12]:
import json

In [13]:
with open("conv2D_input.json", "w") as f:
    json.dump(in_json, f)